In [1]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [4]:
import pandas as pd

In [2]:
# Descargamos el recurso 
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...


True

In [14]:
# Cargo los datos.
df = pd.read_csv(r'Dataset\Procesado\user_reviews.csv')

In [15]:
df.head(3)

,item_id,review
0,1250.0,Simple yet with great replayability. In my opi...
1,22200.0,It's unique and worth a playthrough.
2,43110.0,Great atmosphere. The gunplay can be a bit chu...


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  59305 non-null  float64
 1   review   59275 non-null  object 
dtypes: float64(1), object(1)
memory usage: 927.2+ KB


In [17]:
df['review'] = df['review'].fillna(' ')

In [18]:
#reviso
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59333 entries, 0 to 59332
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  59305 non-null  float64
 1   review   59333 non-null  object 
dtypes: float64(1), object(1)
memory usage: 927.2+ KB


In [19]:
df = df.dropna()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 59332
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   item_id  59305 non-null  float64
 1   review   59305 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.4+ MB


In [21]:
df.describe()

,item_id
count,59305.000000
mean,169935.176511
std,132029.464174
min,10.000000
25%,8930.000000
50%,218230.000000
75%,261760.000000
max,527340.000000


In [22]:
df1 = df.copy()

In [3]:
#Creo la instancia del analizador de sentimiento:
sia = SentimentIntensityAnalyzer()

In [8]:
# Funcion para crear los resultados a etiquetas categoricas

def sentiment_to_label(sentiment_scores):
    if sentiment_scores['compound'] >= 0.05:
        return 2  # Positivo
    elif sentiment_scores['compound'] > -0.05:
        return 1  # Neutral
    else:
        return 0  # Malo

In [23]:
# Aplico el analisis de sentimiento a cada fila y guardo los resultados en una nueva columna del df.
df1['sentiment_analysis'] = df1['review'].apply(sia.polarity_scores).apply(sentiment_to_label)
df1.head(3)

,item_id,review,sentiment_analysis
0,1250.0,Simple yet with great replayability. In my opi...,2
1,22200.0,It's unique and worth a playthrough.,2
2,43110.0,Great atmosphere. The gunplay can be a bit chu...,2


In [25]:
df1.tail(15)

,item_id,review,sentiment_analysis
59318,413150.0,I realy like this game it has gotten me to und...,2
59319,242760.0,"Good Game Play, encountered a problem with cli...",0
59320,427730.0,dont ask,1
59321,280790.0,"Great game, specially if you need to past time...",2
59322,570.0,Good one,2
59323,261030.0,I cried in the end its so sadding ]'; I wish l...,2
59324,730.0,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,1
59325,570.0,Well Done,2
59326,233270.0,this is a very fun and nice 80s themed shooter...,2
59327,130.0,if you liked Half life i would really recommen...,2


In [28]:
#creo las variables dummies para despues poder agrupar por item_id y obtener la suma de todos los reviews.

df_dummies = pd.get_dummies(df1['sentiment_analysis'], prefix='sentiment', dtype='int')

In [29]:
df_dummies.head(3)

,sentiment_0,sentiment_1,sentiment_2
0,0,0,1
1,0,0,1
2,0,0,1


In [31]:
df_dummies = df_dummies.rename(columns={'sentiment_0':'malo'})

In [32]:
df_dummies = df_dummies.rename(columns={'sentiment_1':'neutral'})

In [34]:
df_dummies = df_dummies.rename(columns={'sentiment_2':'positivo'})

In [35]:
#Nombres de columnas cambiados
df_dummies.head(3)

,malo,neutral,positivo
0,0,0,1
1,0,0,1
2,0,0,1


In [44]:
#concateno con el dataframe df1
df_final = pd.concat([df1, df_dummies], axis=1)

In [45]:
df_final

,item_id,review,sentiment_analysis,malo,neutral,positivo
0,1250.0,Simple yet with great replayability. In my opi...,2,0,0,1
1,22200.0,It's unique and worth a playthrough.,2,0,0,1
2,43110.0,Great atmosphere. The gunplay can be a bit chu...,2,0,0,1
3,251610.0,I know what you think when you see this title ...,2,0,0,1
4,227300.0,For a simple (it's actually not all that simpl...,2,0,0,1
...,...,...,...,...,...,...
59328,70.0,a must have classic from steam definitely wort...,2,0,0,1
59329,362890.0,this game is a perfect remake of the original ...,2,0,0,1
59330,273110.0,had so much fun plaing this and collecting res...,2,0,0,1
59331,730.0,:D,2,0,0,1


In [46]:
#ya hecho el analisis elimino las columnas que no nesecito
df_final= df_final.drop(['review', 'sentiment_analysis'], axis=1)

In [47]:
df_final.head(3)

,item_id,malo,neutral,positivo
0,1250.0,0,0,1
1,22200.0,0,0,1
2,43110.0,0,0,1


In [48]:
dff = df_final.copy()

In [54]:
# Agrupo por item_id para tener la sumatoria de reviews
dff = dff.groupby('item_id')[['malo', 'neutral', 'positivo']].sum().reset_index()


In [72]:
dff

,item_id,malo,neutral,positivo
0,10.0,2,21,34
1,20.0,2,5,10
2,30.0,0,1,3
3,40.0,0,0,1
4,50.0,0,0,4
...,...,...,...,...
3677,521340.0,0,0,2
3678,521430.0,0,0,1
3679,521570.0,2,0,0
3680,521990.0,0,0,1


In [79]:
#Cargo el archivo que tenia previamente con lso nombres para hacer un merge de los datos usando al columna item_id
df_names = pd.read_csv(r'Dataset\Procesado\id_games.csv')

In [80]:
df_names = df_names.drop_duplicates()

In [88]:
df_names.groupby('item_id').sum().reset_index()

,item_id,item_name
0,10.0,Counter-Strike
1,20.0,Team Fortress Classic
2,30.0,Day of Defeat
3,40.0,Deathmatch Classic
4,50.0,Half-Life: Opposing Force
...,...,...
10973,528580.0,Found
10974,528660.0,Spin Rush
10975,529670.0,The Bellows: VR Demo
10976,529820.0,Beyond Power VR


In [89]:
df_merged = pd.merge(df_names, dff, on='item_id', how='inner')

In [90]:
df_merged

,item_id,item_name,malo,neutral,positivo
0,10.0,Counter-Strike,2,21,34
1,20.0,Team Fortress Classic,2,5,10
2,30.0,Day of Defeat,0,1,3
3,40.0,Deathmatch Classic,0,0,1
4,50.0,Half-Life: Opposing Force,0,0,4
...,...,...,...,...,...
3316,361330.0,Shrooms,0,1,0
3317,364510.0,Transcendence,0,0,1
3318,427100.0,Fernbus Simulator,0,0,1
3319,367010.0,Vernon's Legacy,0,0,1


In [91]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3321 entries, 0 to 3320
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_id    3321 non-null   float64
 1   item_name  3321 non-null   object 
 2   malo       3321 non-null   int32  
 3   neutral    3321 non-null   int32  
 4   positivo   3321 non-null   int32  
dtypes: float64(1), int32(3), object(1)
memory usage: 90.9+ KB


In [92]:
df_merged.describe()

,item_id,malo,neutral,positivo
count,3321.000000,3321.000000,3321.000000,3321.000000
mean,252774.874737,2.685035,3.199940,10.008130
std,131447.868961,13.580156,21.336032,55.130843
min,10.000000,0.000000,0.000000,0.000000
25%,208670.000000,0.000000,0.000000,1.000000
50%,269270.000000,0.000000,0.000000,1.000000
75%,346940.000000,1.000000,1.000000,5.000000
max,527340.000000,546.000000,946.000000,2267.000000


In [93]:
#Salvo el analisis de los revies por item_id
df_merged.to_csv(r'Dataset\Procesado\items_reviews.csv', index=False)